In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pylab as pl

import os
import sys

import numpy as np
from scipy import ndimage as ni
import maxflow

from tifffile import imread, imsave
import cPickle as pickle

from skimage.filters import gaussian_filter

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

%matplotlib qt

# %matplotlib inline
# %pylab inline
# pylab.rcParams['figure.figsize'] = (5, 5)

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


### Loading image to be segmented

In [2]:
filename = '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF'

sigma = [1,3,3]
image = imread(filename)
image_smooth = gaussian_filter(image,sigma)
print 'Image dimensions: ', image.shape
print 'Image min/max:    ', np.min(image), np.max(image)

Image dimensions:  (31, 512, 512)
Image min/max:     2020 3016


### Load sampled unit sphere and set parameters

In [3]:
plt.rcParams['figure.figsize'] = (5, 5)

# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
num_columns = dictSphereData['n']
col_vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
num_neighbors = len(neighbors)/num_columns
K = 30       # as in Wu & Chen 2002

# net surface related parameters
max_delta_k = 4

# positioning of net in image

# A SWEET SET OF PARAMS
# center_x = 160
# center_y = 190
# center_z = 7
# max_r_x = 40
# max_r_y = 40
# max_r_z = 12

center_x = 110
center_y = 300
center_z = 15
max_r_x = image.shape[2] / 6
max_r_y = image.shape[1] / 6
max_r_z = image.shape[0] / 3
min_r_x = 4
min_r_y = 4
min_r_z = 1

In [4]:
volwin = volshow(image, stackUnits = [1.,1.,10.], raise_window=False)
# volwin = volshow(image_smooth, stackUnits = [1.,1.,4.], raise_window=False)

# Building Graph

### Function definitions

In [5]:
from myfunctions import *

### Compute weights *w*

In [6]:
center = ( center_x, center_y, center_z)
max_radii = ( max_r_x, max_r_y, max_r_z )
min_radii = ( min_r_x, min_r_y, min_r_z )
w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )

### Build and solve flow network

In [7]:
g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
maxval = g.maxflow()

### Visualize results

In [8]:
size_s_comp = 0
size_t_comp = 0
for n in nodes:
    seg = g.get_segment(n)
    if seg == 0:
        size_s_comp += 1
    else:
        size_t_comp += 1
    #print "Segment of the node ",n,":", seg
print "Size of s and t component: ", size_s_comp, size_t_comp

Size of s and t component:  7914 7446


In [9]:
def create_dot_annotatation_image():
    # imgout = np.zeros(image.shape)
    # valtoplace = 1
    imgout = np.copy(image) #np.zeros_like(image)
    valtoplace = 1.0*np.max(image)
    for i in range(num_columns):
        for k in range(K):
            if g.get_segment(i*K+k) == 1: break # leave as soon as k is first outside point
        x = int(center_x + col_vectors[i,0] * min_r_x + col_vectors[i,0] * (k-1)/float(K) * (max_r_x-min_r_x))
        y = int(center_y + col_vectors[i,1] * min_r_y + col_vectors[i,1] * (k-1)/float(K) * (max_r_y-min_r_y))
        z = int(center_z + col_vectors[i,2] * min_r_z + col_vectors[i,2] * (k-1)/float(K) * (max_r_z-min_r_z))
        if x>=0 and y>=0 and z>=0 and \
           x<image.shape[2] and y<image.shape[1] and z<image.shape[0]:
            dc = [[ 0, 0, 0],
                  [ 0,-1, 0],
                  [ 0, 1, 0],
                  [ 0, 0,-1],
                  [ 0, 0, 1]]
            for dz,dy,dx in dc:
                imgout[z+dz,y+dy,x+dx] = valtoplace
    return imgout

# IDEE: draw line to all neighbors (at height k) along 3d bresenham

In [10]:
def create_dot_meshes(facecolor = (1.,.3,.2)):
    meshes = []
    for i in range(num_columns):
        for k in range(K):
            if g.get_segment(i*K+k) == 1: break # leave as soon as k is first outside point
        x = int(center_x + col_vectors[i,0] * min_r_x + col_vectors[i,0] * (k-1)/float(K) * (max_r_x-min_r_x))
        y = int(center_y + col_vectors[i,1] * min_r_y + col_vectors[i,1] * (k-1)/float(K) * (max_r_y-min_r_y))
        z = int(center_z + col_vectors[i,2] * min_r_z + col_vectors[i,2] * (k-1)/float(K) * (max_r_z-min_r_z))
        meshes.append( EllipsoidMesh(rs = norm_radii((1,1,.2),image.shape), pos = norm_coords((x,y,z),image.shape), facecolor = facecolor, alpha=.5) )
    return meshes

In [11]:
def norm_coords(cabs,pixelsizes):
    cnorm = 2. * np.array(cabs[::-1], float) / np.array(pixelsizes) - 1.
    return tuple(cnorm[::-1])

def norm_radii(cabs,pixelsizes):
    cnorm = 2. * np.array(cabs[::-1], float) / np.array(pixelsizes)
    return tuple(cnorm[::-1])

print norm_coords((200,100,20), image.shape)

(-0.21875, -0.609375, 0.29032258064516125)


In [12]:
from spimagine import EllipsoidMesh, Mesh
def create_surface_mesh(center, g, facecolor=(1.,.3,.2)):
#     from spimagine import Mesh
#     mesh = Mesh(vertices=myverts, normals = mynormals, indices = myinds)
#     w.glWidget.add_mesh(mesh)
#     vertices und normals sind die Punkte/Normalen und haben shape = (n_points, 3),
#     indices ist eine int array der laenge (3*n_triangles) mit den indizies der Triangles 
#     (d.h. indices = [0,1,2] baut ein Dreieck aus vertices[0], [1] und [2])
    myverts = np.zeros((num_columns, 3))
    mynormals = col_vectors
    myinds = np.zeros(num_columns*3) # NEEDS IMPROVEMENT!!!
    for i in range(num_columns):
        for k in range(K):
            if g.get_segment(i*K+k) == 1: break # leave as soon as k is first outside point
        x = int(center[0] + col_vectors[i,0] * min_r_x + col_vectors[i,0] * (k-1)/float(K) * (max_r_x-min_r_x))
        y = int(center[1] + col_vectors[i,1] * min_r_y + col_vectors[i,1] * (k-1)/float(K) * (max_r_y-min_r_y))
        z = int(center[2] + col_vectors[i,2] * min_r_z + col_vectors[i,2] * (k-1)/float(K) * (max_r_z-min_r_z))
        myverts[i,:] = norm_coords([x,y,z],image.shape)
        myinds[3*i]   = i
        hood = neighbors_of[i]
        for j in range(len(hood)):
            myinds[3*i+1] = neighbors_of[i][j]
            myinds[3*i+2] = neighbors_of[i][(j+1)%len(hood)]
    return Mesh(vertices=myverts, normals = mynormals, indices = myinds, facecolor=facecolor, alpha=.5)

print create_surface_mesh(center, g)

In [13]:
from spimagine import EllipsoidMesh, Mesh
def create_dot_pyramids(center, g, facecolor=(1.,.3,.2)):
#     from spimagine import Mesh
#     mesh = Mesh(vertices=myverts, normals = mynormals, indices = myinds)
#     w.glWidget.add_mesh(mesh)
#     vertices und normals sind die Punkte/Normalen und haben shape = (n_points, 3),
#     indices ist eine int array der laenge (3*n_triangles) mit den indizies der Triangles 
#     (d.h. indices = [0,1,2] baut ein Dreieck aus vertices[0], [1] und [2])
    myverts = np.zeros((num_columns*4, 3))
    mynormals = np.zeros((num_columns*4, 3))
    myinds = np.zeros(num_columns*12)
    for i in range(num_columns):
        for k in range(K):
            if g.get_segment(i*K+k) == 1: break # leave as soon as k is first outside point
        x = int(center[0] + col_vectors[i,0] * min_r_x + col_vectors[i,0] * (k-1)/float(K) * (max_r_x-min_r_x))
        y = int(center[1] + col_vectors[i,1] * min_r_y + col_vectors[i,1] * (k-1)/float(K) * (max_r_y-min_r_y))
        z = int(center[2] + col_vectors[i,2] * min_r_z + col_vectors[i,2] * (k-1)/float(K) * (max_r_z-min_r_z))
        
        mynormals[3*i,:]   = col_vectors[i]
        mynormals[3*i+1,:] = col_vectors[i]
        mynormals[3*i+2,:] = col_vectors[i]
        mynormals[3*i+3,:] = col_vectors[i]
        
        myverts[3*i,:]   = norm_coords([x-.5,y+.5,z+.5],image.shape)
        myverts[3*i+1,:] = norm_coords([x+.5,y+.5,z-.5],image.shape)
        myverts[3*i+2,:] = norm_coords([x-.5,y-.5,z-.5],image.shape)
        myverts[3*i+3,:] = norm_coords([x+.5,y-.5,z+.5],image.shape)
        
        myinds[3*i]   = 3*i
        myinds[3*i+1] = 3*i+1
        myinds[3*i+2] = 3*i+2
        myinds[3*i+3] = 3*i
        myinds[3*i+4] = 3*i+1
        myinds[3*i+5] = 3*i+3
        myinds[3*i+6] = 3*i+2
        myinds[3*i+7] = 3*i+3
        myinds[3*i+8] = 3*i
        myinds[3*i+9] = 3*i+2
        myinds[3*i+10]= 3*i+3
        myinds[3*i+11]= 3*i+1
    return Mesh(vertices=myverts, normals = mynormals, indices = myinds, facecolor=facecolor)

print create_surface_mesh()

TypeError: create_surface_mesh() takes at least 2 arguments (0 given)

In [16]:
volwin = volshow(create_dot_annotatation_image(), stackUnits = [1.,1.,5.], raise_window=False)
mesh_min = EllipsoidMesh(rs = norm_radii((3,3,.5),image.shape), pos = norm_coords(center,image.shape), facecolor = (1.,.3,.2), alpha=.5)
volwin.glWidget.add_mesh(mesh_min)
# mesh_min = EllipsoidMesh(rs = norm_radii(min_radii,image.shape), pos = norm_coords(center,image.shape), facecolor = (1.,.3,.2), alpha=.5)
# volwin.glWidget.add_mesh(mesh_min)
# mesh_max = EllipsoidMesh(rs = norm_radii(max_radii,image.shape), pos = norm_coords(center,image.shape), facecolor = (.2,1.,.3), alpha=.5)
# volwin.glWidget.add_mesh(mesh_max)
# volwin.glWidget.add_mesh( create_surface_mesh() )

In [17]:
volwin = volshow(image, stackUnits = [1.,1.,5.], raise_window=False)
meshes = create_dot_meshes()
for m in meshes:
    volwin.glWidget.add_mesh(m)

In [30]:
volwin = volshow(image, stackUnits = [1.,1.,5.], raise_window=False)
volwin.glWidget.add_mesh( create_dot_pyramids(facecolor=(.4,.4,1.)) )

# Create video frames 

In [103]:
frames = []
f = range(100,431,10)
for i, center_x in enumerate(f):
    
    center_y = 280
    center_z = 26
    center = ( center_x, center_y, center_z)

    w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )
    g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
    maxval = g.maxflow()
    
    frames.append( create_dot_annotatation_image() )
    print '.',
    
f = range(280,200,-10)
for i, center_y in enumerate(f):
    
    center_x = 430
    center_z = 26
    center = ( center_x, center_y, center_z)

    w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )
    g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
    maxval = g.maxflow()
    
    frames.append( create_dot_annotatation_image() )
    print '.',

print 'done'

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done


In [83]:
volwin = volshow(np.array(frames), stackUnits = [1.,1.,4.], raise_window=True)

/Users/jug/Repositories/GIT/BobSeg/video


In [99]:
imsave('video/center_sampling_maddox_spim_dots.tif', np.array(frames))

# Multi-cell stuff 

In [14]:
def segment_and_plot(center, col=(.4,1.,.4) ):
    max_radii = ( max_r_x, max_r_y, max_r_z )
    min_radii = ( min_r_x, min_r_y, min_r_z )
    w, w_tilde = compute_weights( image, center, K, max_radii, col_vectors, min_radii=min_radii )
    g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde )
    maxval = g.maxflow()
    
#     volwin.glWidget.add_mesh( create_dot_pyramids(center, g, facecolor=col) )
    volwin.glWidget.add_mesh( create_surface_mesh(center, g, facecolor=col) )
    mesh_center = EllipsoidMesh(rs=norm_radii((3,3,.5),image.shape), pos=norm_coords(center,image.shape), facecolor=col, alpha=.5)
    volwin.glWidget.add_mesh(mesh_center)

    return g,nodes,w,w_tilde

In [15]:
volwin = volshow(image, stackUnits = [1.,1.,5.], raise_window=False)

green1 = (.7,1.,.3)
green2 = (.3,1.,.7)

orange = (.5,.3,.2)
red = (.5,.2,.2)
# orange = (.8,.6,.4)
# red = (1.,.4,.4)

# E cells
# -------
c = (center_x, center_y+50, center_z)
col = green1
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x, center_y, center_z)
col = green2
g, nodes, w, w_tilde = segment_and_plot(c,col)

# Neighbors
# ---------
c = (center_x-20, center_y+90, center_z)
col = red
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x+40, center_y+110, center_z)
col = orange
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x+90, center_y+90, center_z)
col = red
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x+80, center_y+15, center_z)
col = orange
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x+35, center_y-35, center_z)
col = red
g, nodes, w, w_tilde = segment_and_plot(c,col)

c = (center_x, center_y-70, center_z)
col = orange
g, nodes, w, w_tilde = segment_and_plot(c,col)

# c = (center_x, center_y-180, center_z)
# col = (1.,.4,.4)
# g, nodes, w, w_tilde = segment_and_plot(c,col)